In [8]:
import torch
from transformers import LlamaModel, LlamaTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import  AutoModelForCausalLM, GenerationConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from pathlib import Path
from transformers import AutoTokenizer
from torch.nn.utils.rnn import pad_sequence
from peft import PeftModel
import evaluate
import numpy as np
from accelerate import Accelerator
from transformers import TrainerCallback 


# Load GSM8k dataset
dataset = load_dataset("gsm8k", "main")

# Print the first few examples
print(dataset["train"][0:5])

{'question': ['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?', 'Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?', 'Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?', 'James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?'], 'answer': ['Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether i

In [10]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    # 'gpt2',
    "NousResearch/Llama-2-7b-hf", 
    trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

# model stuff
model = AutoModelForCausalLM.from_pretrained(
    'NousResearch/Llama-2-7b-hf',
    # 'gpt2', 
    use_flash_attention_2=True, 
    torch_dtype=torch.float16, 
    trust_remote_code =True,
    do_sample=False)
model.train()

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:27<00:00, 13.55s/it]
/home/asetlur/anaconda3/envs/syth-llm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/asetlur/anaconda3/envs/syth-llm/lib/python3.10/site-packages/transformers/gene

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()

In [20]:
print(dataset['train'][0]['question'])
print(dataset['train'][0]['answer'])

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72


In [2]:
10898853453 / 345769

31520.6205674887

In [18]:
# Calculate the likelihood using the Llama-7B model

inputs = torch.LongTensor(tokenizer.encode(dataset['train'][0]['answer']))
likelihood = model(inputs).logits

# print(likelihood)


ValueError: not enough values to unpack (expected 3, got 2)